# Imports and Data Directory

In [1]:
from __future__ import print_function
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from sklearn.metrics import classification_report, confusion_matrix, f1_score
import os
import pandas as pd
import numpy as np
import shutil
import zipfile

# Delete data directory if present.
shutil.rmtree('ECEN489Py4Data', ignore_errors=True)

# Extract data.
with zipfile.ZipFile("ECEN489Py4Data.zip", "r") as zip_ref:
    zip_ref.extractall('ECEN489Py4Data')

# Set directory for data.
data_dir = os.path.join('ECEN489Py4Data', 'data')

Using TensorFlow backend.


# Image Data Generation

In [2]:
#ImageDataGenerator implements functions useful for input image scaling and augmentation -- you may want more!

train_datagen = ImageDataGenerator(rescale=1./255)

valid_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)


In [3]:
train_generator = train_datagen.flow_from_directory(
        os.path.join(data_dir, 'train'),
        target_size=(32, 32),
        color_mode='rgb',
        batch_size=1,
        class_mode='categorical',
        shuffle=True,
        seed=1953)


valid_generator = valid_datagen.flow_from_directory(
        os.path.join(data_dir, 'validation'),
        target_size=(32, 32),
        color_mode='rgb',
        batch_size=1,
        class_mode='categorical',
        shuffle=False, #don't shuffle or label results will be all wrong
        seed=1953)


test_generator = test_datagen.flow_from_directory(
    os.path.join(data_dir, 'test'),
    target_size=(32, 32),
    color_mode='rgb',
    batch_size=1,
    class_mode='categorical',
    shuffle=False, #don't shuffle or label results will be all wrong
    seed=1953)

STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size

Found 2744 images belonging to 8 classes.
Found 929 images belonging to 8 classes.
Found 0 images belonging to 0 classes.


# Function for Presenting Model Results

In [4]:
def present_results(m, g, steps):
    Y_pred = m.predict_generator(g, steps=steps)
    y_pred = np.argmax(Y_pred, axis=1)
    print('Confusion Matrix')
    print(confusion_matrix(g.classes, y_pred))
    print('Classification Report')
    print(classification_report(g.classes, y_pred, target_names=g.class_indices))
    print("F1 score (using average='micro')")
    print(f1_score(y_true=g.classes, y_pred=y_pred, average='micro'))

# CNN From Nowka

In [5]:
# neural network model
# you may want to vary these parameters, etc

num_classes = 8 # fixed by the number of classes of signs that we gave you. Dont change

# model = Sequential()
# model.add(Conv2D(32, (3, 3), padding='same',input_shape = (32, 32, 3)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Conv2D(64, (3, 3), padding='same'))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Flatten())
# model.add(Dense(512))
# model.add(Activation('relu'))
# model.add(Dropout(0.25))
# model.add(Dense(num_classes))
# model.add(Activation('softmax'))

# model.compile(loss = 'categorical_crossentropy',
#               optimizer = 'Adam', # may want to try others
#               metrics = ['accuracy'])
# model.summary()

# # Train and save.
# model.fit_generator(generator=train_generator,
#                     steps_per_epoch=STEP_SIZE_TRAIN,
#                     validation_data=valid_generator,
#                     validation_steps=STEP_SIZE_VALID,
#                     epochs=10 # may need to increase if not seeing low enough losses
# )
# model.save('cnn_nowka.h5')
# del model

# Part 2 Results

In [6]:
# NOTE: Model has been saved, loading it and making predictions.
present_results(m=keras.models.load_model('cnn_nowka.h5'),
                g=valid_generator, steps=STEP_SIZE_VALID)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Confusion Matrix
[[ 52   0   0   0   0   0   0   0]
 [  0  49   1   0   0   0   0   0]
 [  0   1 195   2   0   0   2   0]
 [  0   0   2 148   0   0   4   0]
 [  0   0   0   0  17   0   0   0]
 [  0   0   0   0   0  47   0   0]
 [  1   0   2   1   0   0 383   0]
 [  0   0   0   0   0   1   3  18]]
Classification Report
              precision    recall  f1-score   support

       merge       0.98      1.00      0.99        52
  pedestrian       0.98      0.98      0.98        50
       yield       0.97      0.97      0.97       200
        stop       0.98      0.96      0.97       154
   keepRight       1.00      1.00      1.00        17
 signalAhead       0.98      1.00      0.99        47
speedLimit25       0.98      0.99      0.98       387
speedLimit35

In [7]:
#USE CELLS BELOW TO COMPLETE THE EXERCISE WITH THE TEST SET

In [8]:
# STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
# present_results(m=model, g=test_generator, steps=STEP_SIZE_TEST)

In [9]:
#This cell dumps out a file of which files were incorrectly predicted
#so you can see if you need more features, more training samples, etc
# predicted_class_indices=np.argmax(Y_pred,axis=1)
# labels = (train_generator.class_indices)
# labels = dict((v,k) for k,v in labels.items())
# predictions = [labels[k] for k in predicted_class_indices]
# # NOTE/TODO: Change this to test_generator later.
# filenames=valid_generator.filenames
# # filenames=test_generator.filenames
# print(len(filenames))
# print(len(predictions))
# results=pd.DataFrame({"Filename":filenames,
#                       "Predictions":predictions})
# results.to_csv("results.csv",index=False)

# Part 3

## Delete bad data from training

In [10]:
# The following are in 'yield' but are actually 'merge'
bad = [
    '140_yield_1323813350.avi_image2.png',
    '141_yield_1323813350.avi_image3.png',
    '142_yield_1323813350.avi_image4.png',
    '143_yield_1323813350.avi_image5.png',
    '144_yield_1323813350.avi_image6.png',
    '146_yield_1323813350.avi_image8.png',
    '192_yield_1323816786.avi_image1.png',
    '193_yield_1323816786.avi_image10.png',
    '194_yield_1323816786.avi_image11.png',
    '195_yield_1323816786.avi_image12.png',
    '196_yield_1323816786.avi_image13.png',
    '199_yield_1323816786.avi_image16.png',
    '204_yield_1323816786.avi_image20.png',
    '206_yield_1323816786.avi_image22.png',
    '207_yield_1323816786.avi_image23.png',
    '208_yield_1323816786.avi_image24.png',
    '209_yield_1323816786.avi_image25.png',
    '212_yield_1323816786.avi_image5.png',
    '213_yield_1323816786.avi_image6.png',
    '214_yield_1323816786.avi_image7.png',
    '215_yield_1323816786.avi_image8.png',
    '257_yield_1323821570.avi_image0.png',
    '258_yield_1323821570.avi_image1.png',
    '261_yield_1323821570.avi_image4.png',
    '262_yield_1323821570.avi_image5.png',
    '264_yield_1323821570.avi_image7.png',
    '265_yield_1323821570.avi_image8.png',
    
]

train_dir = os.path.join(data_dir, 'train')

for f in bad:
    os.rename(os.path.join(train_dir, 'yield', f), os.path.join(train_dir, 'merge', f))

## Recreate Image Data Generator(s)

In [11]:
# Use some rotations, shears, and flips 
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=30, shear_range=30,
                                   horizontal_flip=True, vertical_flip=True)

train_generator = train_datagen.flow_from_directory(
        os.path.join(data_dir, 'train'),
        target_size=(32, 32),
        color_mode='rgb',
        batch_size=1,
        class_mode='categorical',
        shuffle=True,
        seed=1953)


Found 2744 images belonging to 8 classes.


## Early Stopping

In [12]:
early_stop = keras.callbacks.EarlyStopping(monitor='val_acc', patience=10,
                                          restore_best_weights=True)

## CNN 1 (deeper)

In [13]:
# Use more filters and one extra convolutional layer.
# cnn = Sequential()
# # Conv 1.
# cnn.add(Conv2D(64, (3, 3), padding='same',input_shape = (32, 32, 3)))
# cnn.add(Activation('relu'))
# cnn.add(MaxPooling2D(pool_size=(2, 2)))
# # Conv 2.
# cnn.add(Conv2D(128, (3, 3), padding='same'))
# cnn.add(Activation('relu'))
# cnn.add(MaxPooling2D(pool_size=(2, 2)))
# # Conv 3.
# cnn.add(Conv2D(256, (2, 2), padding='same'))
# cnn.add(Activation('relu'))
# cnn.add(MaxPooling2D(pool_size=(2, 2)))
# # Flatten.
# cnn.add(Flatten())
# # Dense.
# cnn.add(Dense(512))
# cnn.add(Activation('relu'))
# cnn.add(Dropout(0.25))
# # Predict.
# cnn.add(Dense(num_classes))
# cnn.add(Activation('softmax'))

# # Print summary, compile.
# cnn.summary()
# cnn.compile(loss=keras.losses.categorical_crossentropy, optimizer='adam', metrics=['accuracy'])

# # Train.
# cnn.fit_generator(generator=train_generator,
#                     steps_per_epoch=STEP_SIZE_TRAIN,
#                     validation_data=valid_generator,
#                     validation_steps=STEP_SIZE_VALID,
#                     epochs=100,
#                     callbacks=[early_stop]
# )

# # Save.
# cnn.save('cnn_deeper.h5')

# # Clear.
# del cnn

## CNN 2 (shallower)

In [14]:
# # Use larger initial convolution layer with more filters.
# cnn = Sequential()
# # Conv 1.
# cnn.add(Conv2D(128, (4, 4), padding='same',input_shape = (32, 32, 3)))
# cnn.add(Activation('relu'))
# cnn.add(MaxPooling2D(pool_size=(2, 2)))
# # Conv 2.
# cnn.add(Conv2D(256, (2, 2), padding='same'))
# cnn.add(Activation('relu'))
# cnn.add(MaxPooling2D(pool_size=(2, 2)))
# # Flatten.
# cnn.add(Flatten())
# # Dense.
# cnn.add(Dense(512))
# cnn.add(Activation('relu'))
# cnn.add(Dropout(0.25))
# # Predict.
# cnn.add(Dense(num_classes))
# cnn.add(Activation('softmax'))

# # Print summary, compile.
# cnn.summary()
# cnn.compile(loss=keras.losses.categorical_crossentropy, optimizer='adam', metrics=['accuracy'])

# # Train.
# cnn.fit_generator(generator=train_generator,
#                     steps_per_epoch=STEP_SIZE_TRAIN,
#                     validation_data=valid_generator,
#                     validation_steps=STEP_SIZE_VALID,
#                     epochs=100,
#                     callbacks=[early_stop]
# )

# # Save.
# cnn.save('cnn_shallow.h5')

# # Clear.
# del cnn

## Results

### Validation

In [15]:
# NOTE: models have been saved. Loading them up and making predictions.
print('Deeper:')
present_results(m=keras.models.load_model('cnn_deeper.h5'),
                g=valid_generator, steps=STEP_SIZE_VALID)
print('*'*80)
print('Shallower:')
present_results(m=keras.models.load_model('cnn_shallow.h5'),
                g=valid_generator, steps=STEP_SIZE_VALID)

Deeper:
Confusion Matrix
[[ 39   7   3   1   0   1   0   1]
 [  0  37  12   1   0   0   0   0]
 [  0   2 187  11   0   0   0   0]
 [  0   0   3 137   6   6   2   0]
 [  0   0   0   3   3  11   0   0]
 [  0   0   0   1   1  34  11   0]
 [  0   0   4   4   0   1 367  11]
 [ 11   0   0   0   0   0   4   7]]
Classification Report
              precision    recall  f1-score   support

       merge       0.78      0.75      0.76        52
  pedestrian       0.80      0.74      0.77        50
       yield       0.89      0.94      0.91       200
        stop       0.87      0.89      0.88       154
   keepRight       0.30      0.18      0.22        17
 signalAhead       0.64      0.72      0.68        47
speedLimit25       0.96      0.95      0.95       387
speedLimit35       0.37      0.32      0.34        22

   micro avg       0.87      0.87      0.87       929
   macro avg       0.70      0.69      0.69       929
weighted avg       0.87      0.87      0.87       929

F1 score (using avera

### Testing

In [16]:
# present_results(m=cnn, g=test_generator, steps=STEP_SIZE_TEST)